In [82]:
#TO DO

#Merge base dataset to create a Nom du point d'arrêt in French
#Aggregate by Heure de pointe du soir and Heure de pointe du matin Matin : 6h – 9h Soir : 16h – 19h
#Download in bulk data from septembre 2022 for comparison


import pandas as pd

# Load the CSV file into a DataFrame
# df = pd.read_csv(r'C:\Users\ambc\Documents\trains.csv', sep=',', )

df = pd.read_csv(r'C:\Users\ambc\Documents\trains_29_08.csv', sep=';', )

df = df[['Date de départ', 'Numéro du train', 'Relation', 'Opérateur', 
                       'Heure réelle de départ', "Heure réelle d'arrivée", 'Retard au départ', 'Heure prévue de départ', "Direction de la relation", "Nom du point d'arrêt"]]

df = df[df['Opérateur'] == 'SNCB/NMBS']

# Display the first few rows of the dataset
df.head()

,Date de départ,Numéro du train,Relation,Opérateur,Heure réelle de départ,Heure réelle d'arrivée,Retard au départ,Heure prévue de départ,Direction de la relation,Nom du point d'arrêt
0,2023-08-29,14,ICE,SNCB/NMBS,17:49:36,17:49:36,4116.0,16:41:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ANGLEUR
1,2023-08-29,14,ICE,SNCB/NMBS,17:58:39,17:58:39,4059.0,16:51:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ANS
2,2023-08-29,14,ICE,SNCB/NMBS,18:21:15,18:21:15,4035.0,17:14:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,VELTEM
3,2023-08-29,14,ICE,SNCB/NMBS,18:24:08,18:24:08,4028.0,17:17:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,NOSSEGEM
4,2023-08-29,14,ICE,SNCB/NMBS,18:24:59,18:24:59,4019.0,17:18:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ZAVENTEM


In [83]:
#Cleaning
# Filter out rows where 'Heure réelle de départ' and 'Heure réelle d'arrivée' are the same so that we get rid of
#points d'arrêt that are not stations

df = df[df['Heure réelle de départ'] != df['Heure réelle d\'arrivée']]

# Remove rows where 'Relation' contains the values 'TGV' or 'ICE'
df = df[~df['Relation'].isin(['TGV', 'ICE'])]

# Extracting 'Départ de la ligne' and 'Arrivée de la ligne' from 'Direction de la relation' column
departure = df['Direction de la relation'].str.split('->').str[0].str.split(':').str[1].str.strip()
arrival = df['Direction de la relation'].str.split('->').str[1].str.strip()

# Updating the copied dataframe using .loc
df.loc[:, 'Départ de la ligne'] = departure
df.loc[:, 'Arrivée de la ligne'] = arrival

#Drop columns qui sert plus à rien
df = df.drop(columns=['Heure réelle d\'arrivée', 'Direction de la relation'])

# Remove rows where 'Heure réelle de départ' has no value (NaN)
#df = df.dropna(subset=['Heure réelle de départ'])



In [84]:
# Select only weekdays from Monday to Friday

# Convert the "Date de départ" column to datetime format
df['Date de départ'] = pd.to_datetime(df['Date de départ'])

# Filter rows where the weekday is between Monday (0) and Friday (4)
df = df[df['Date de départ'].dt.weekday.between(0, 4)]



In [85]:
df = df.sort_values(by="Retard au départ", ascending=False)
df.head(10)

,Date de départ,Numéro du train,Relation,Opérateur,Heure réelle de départ,Retard au départ,Heure prévue de départ,Nom du point d'arrêt,Départ de la ligne,Arrivée de la ligne
38303,2023-08-29,9271,IC 35,SNCB/NMBS,23:57:15,6975.0,22:01:00,NOORDERKEMPEN,BRUSSEL-ZUID,AMSTERDAM CENTRAAL
20479,2023-08-29,3580,L B5-2,SNCB/NMBS,11:41:18,3618.0,10:41:00,HUIZINGEN,MECHELEN,EDINGEN
20477,2023-08-29,3580,L B5-2,SNCB/NMBS,11:36:14,3554.0,10:37:00,BEERSEL,MECHELEN,EDINGEN
58652,2023-08-29,3580,L B5-2,SNCB/NMBS,11:31:46,3466.0,10:34:00,MOENSBERG,MECHELEN,EDINGEN
58651,2023-08-29,3580,L B5-2,SNCB/NMBS,11:24:37,3397.0,10:28:00,DIESDELLE,MECHELEN,EDINGEN
20475,2023-08-29,3580,L B5-2,SNCB/NMBS,11:26:58,3358.0,10:31:00,SINT-JOB,MECHELEN,EDINGEN
21292,2023-08-29,3580,L B5-2,SNCB/NMBS,11:21:21,3321.0,10:26:00,BOONDAAL,MECHELEN,EDINGEN
21290,2023-08-29,3580,L B5-2,SNCB/NMBS,11:16:53,3293.0,10:22:00,ETTERBEEK,MECHELEN,EDINGEN
24494,2023-08-29,3217,IC 26,SNCB/NMBS,18:31:17,3197.0,17:38:00,KORTRIJK,KORTRIJK,SINT-NIKLAAS
7309,2023-08-29,2181,L B4,SNCB/NMBS,11:16:03,2823.0,10:29:00,MOUTERIJ,AALST,MECHELEN


In [28]:
# Filter the dataframe for trains from ANTWERPEN-CENTRAAL to HASSELT
antwerpen_to_hasselt_df = df[
    (df['Départ de la ligne'] == 'ANTWERPEN-CENTRAAL') &
    (df['Arrivée de la ligne'] == 'HASSELT')
]

antwerpen_to_hasselt_df

,Date de départ,Numéro du train,Relation,Opérateur,Heure réelle de départ,Heure réelle d'arrivée,Retard à l'arrivée,Retard au départ,Direction de la relation,Nom du point d'arrêt,Départ de la ligne,Arrivée de la ligne
9861,2023-08-29,2606,IC 08,SNCB/NMBS,06:50:02,NaN,NaN,62.0,IC 08: ANTWERPEN-CENTRAAL -> HASSELT,ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,HASSELT
9867,2023-08-29,2606,IC 08,SNCB/NMBS,07:08:37,07:06:39,39.0,37.0,IC 08: ANTWERPEN-CENTRAAL -> HASSELT,MECHELEN,ANTWERPEN-CENTRAAL,HASSELT
9877,2023-08-29,2606,IC 08,SNCB/NMBS,08:09:08,08:06:38,-81.0,8.0,IC 08: ANTWERPEN-CENTRAAL -> HASSELT,DIEST,ANTWERPEN-CENTRAAL,HASSELT
9945,2023-08-29,2607,IC 08,SNCB/NMBS,07:49:59,NaN,NaN,59.0,IC 08: ANTWERPEN-CENTRAAL -> HASSELT,ANTWERPEN-CENTRAAL,ANTWERPEN-CENTRAAL,HASSELT
9953,2023-08-29,2607,IC 08,SNCB/NMBS,08:56:09,08:54:29,29.0,9.0,IC 08: ANTWERPEN-CENTRAAL -> HASSELT,AARSCHOT,ANTWERPEN-CENTRAAL,HASSELT
...,...,...,...,...,...,...,...,...,...,...,...,...
66900,2023-08-29,4916,IC 15,SNCB/NMBS,16:35:37,16:32:52,-7.0,37.0,IC 15: ANTWERPEN-CENTRAAL -> HASSELT,LIER,ANTWERPEN-CENTRAAL,HASSELT
66907,2023-08-29,4918,IC 15,SNCB/NMBS,18:23:51,18:22:07,67.0,51.0,IC 15: ANTWERPEN-CENTRAAL -> HASSELT,ANTWERPEN-BERCHEM,ANTWERPEN-CENTRAAL,HASSELT
66914,2023-08-29,4918,IC 15,SNCB/NMBS,NaN,19:20:18,198.0,NaN,IC 15: ANTWERPEN-CENTRAAL -> HASSELT,HASSELT,ANTWERPEN-CENTRAAL,HASSELT
67016,2023-08-29,4919,IC 15,SNCB/NMBS,19:47:00,19:44:47,-12.0,60.0,IC 15: ANTWERPEN-CENTRAAL -> HASSELT,HEIST-OP-DEN-BERG,ANTWERPEN-CENTRAAL,HASSELT


In [ ]:
df = df[df['Numéro du train'] == 2606]
df

In [15]:
df.to_csv('data_train_cleaned.csv', index=False)